# HW 9 TWO HEADS VAR1

In [1]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import tensorflow as tf
import os

In [2]:
path_to_file = 'evgenyi_onegin.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

text = text.split('\n\n')

In [3]:
text[6]

'                        "Мой дядя самых честных правил,\n                        Когда не в шутку занемог,\n                        Он уважать себя заставил\n                        И лучше выдумать не мог.\n                        Его пример другим наука;\n                        Но, боже мой, какая скука\n                        С больным сидеть и день и ночь,\n                        Не отходя ни шагу прочь!\n                        Какое низкое коварство\n                        Полуживого забавлять,\n                        Ему подушки поправлять,\n                        Печально подносить лекарство,\n                        Вздыхать и думать про себя:\n                        Когда же черт возьмет тебя!"'

In [4]:
len(text)

782

In [5]:
text_only = []
for strofa in text:
    if len(strofa) < 350:
        continue
    else:
        text_only.append(strofa)
len(text_only)

376

In [6]:
text_only[155]

'                        Кого ж любить? Кому же верить?\n                        Кто не изменит нам один?\n                        Кто все дела, все речи мерит\n                        Услужливо на наш аршин?\n                        Кто клеветы про нас не сеет?\n                        Кто нас заботливо лелеет?\n                        Кому порок наш не беда?\n                        Кто не наскучит никогда?\n                        Призрака суетный искатель,\n                        Трудов напрасно не губя,\n                        Любите самого себя,\n                        Достопочтенный мой читатель!\n                        Предмет достойный: ничего\n                        Любезней, верно, нет его.'

In [7]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "Content"})
data

,Content
0,Не мысля гордый свет з...
1,"""Мой дядя самых честны..."
2,Так думал молодой пове...
3,Служив отлично благоро...
4,Когда же юности мятежн...
...,...
371,А счастье было так воз...
372,Она ушла. Стоит Евгени...
373,"Кто б ни был ты, о мой..."
374,"Прости ж и ты, мой спу..."


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()




def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)    
    txt = re.sub("\n", " \n", txt)    
    return txt



def preprocess_text(txt, morph = False):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\n", "zzz", txt)
    new_txt =[]
    for word in txt.split():
        if word == "zzz":
            word = " \n"
            
        else:
            if morph:
                word = morpher.parse(word)[0].normal_form
            else:
                pass
        new_txt.append(word)

    return new_txt


data['Content_splited'] = data['Content'].apply(exclude_punctuation)


data['Content_splited_morph'] = data['Content_splited'].apply(preprocess_text, morph = True)
data['Content_splited'] = data['Content_splited'].apply(preprocess_text, morph = False)

data

,Content,Content_splited,Content_splited_morph
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима...","[не, мыслить, гордый, свет, забавить, \n, вни..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда...","[мой, дядя, самый, честной, правило, \n, когд..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы...","[так, думать, молодой, повеса, \n, лететь, в,..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи...","[служивый, отлично, благородно, \n, долг, жит..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг...","[когда, же, юность, мятежный, \n, прийти, евг..."
...,...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл...","[а, счастие, быть, так, возможно, \n, так, бл..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г...","[она, уйти, стоить, евгений, \n, как, будто, ..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д...","[кто, б, ни, быть, ты, о, мой, читатель, \n, ..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n...","[простить, ж, и, ты, мой, спутник, странный, ..."


In [9]:
def get_w2i_i2w(column_data):
    
    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))
#    print('{} unique characters'.format(len(vocab)))            
            
    # Creating a mapping from unique characters to indices
    word2idx = {u:i for i, u in enumerate(vocab)}
    idx2word = np.array(vocab)
    
    print(len(vocab), len(word2idx), len(idx2word))
    return word2idx, idx2word
    

word2idx, idx2word = get_w2i_i2w(data['Content_splited'])    
word2idx_morph, idx2word_morph = get_w2i_i2w(data['Content_splited_morph'])    
    
data['int_Content_splited'] = data['Content_splited'].apply(lambda x: [word2idx[c] for c in x])
data['int_Content_splited_morph'] = data['Content_splited_morph'].apply(lambda x: [word2idx_morph[c] for c in x])            
            

8427 8427 8427
5047 5047 5047


In [10]:
data

,Content,Content_splited,Content_splited_morph,int_Content_splited,int_Content_splited_morph
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима...","[не, мыслить, гордый, свет, забавить, \n, вни...","[3817, 3634, 1358, 6327, 2071, 0, 878, 1844, 9...","[2253, 2140, 844, 3827, 1222, 0, 556, 1095, 58..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда...","[мой, дядя, самый, честной, правило, \n, когд...","[3487, 1912, 6292, 8129, 5394, 0, 2788, 3817, ...","[2064, 1134, 3798, 4875, 3259, 0, 1664, 2253, ..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы...","[так, думать, молодой, повеса, \n, лететь, в,...","[7249, 1867, 3505, 4930, 0, 3102, 565, 5872, 3...","[4324, 1110, 2077, 2953, 0, 1860, 384, 3553, 2..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи...","[служивый, отлично, благородно, \n, долг, жит...","[6660, 4535, 325, 0, 1720, 2041, 1919, 4512, 0...","[4007, 2710, 254, 0, 1039, 1204, 2602, 2692, 0..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг...","[когда, же, юность, мятежный, \n, прийти, евг...","[2788, 1980, 8363, 3638, 0, 5651, 1915, 5222, ...","[1664, 1168, 5011, 2144, 0, 3352, 1135, 3151, ..."
...,...,...,...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл...","[а, счастие, быть, так, возможно, \n, так, бл...","[98, 7202, 550, 7249, 924, 0, 7249, 380, 4072,...","[98, 4295, 383, 4324, 584, 0, 4324, 279, 2421,..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г...","[она, уйти, стоить, евгений, \n, как, будто, ...","[4371, 7870, 7006, 1914, 0, 2637, 515, 1462, 5...","[2603, 4579, 4199, 1135, 0, 1571, 361, 899, 31..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д...","[кто, б, ни, быть, ты, о, мой, читатель, \n, ...","[2960, 160, 4037, 546, 7548, 4141, 3487, 8153,...","[1780, 151, 2398, 383, 4493, 2453, 2064, 4888,..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n...","[простить, ж, и, ты, мой, спутник, странный, ...","[5751, 1951, 2446, 7548, 3487, 6916, 7049, 0, ...","[3477, 1153, 1454, 4493, 2064, 4156, 4222, 0, ..."


In [11]:
i = 100
len(data.iloc[i]['int_Content_splited']), len(data.iloc[i]['int_Content_splited_morph'])
#data.iloc[i]['int_Content_splited_morph']

(68, 68)

In [12]:
def get_all_int(column_data):
    
    int_dump = column_data.values
    all_txt_as_int = []

    for sublist in int_dump:
        for item in sublist:
            all_txt_as_int.append(item)
    all_txt_as_int = np.array(all_txt_as_int)    
    
    return all_txt_as_int

all_txt_as_int = get_all_int(data['int_Content_splited'])

all_txt_as_int_morph = get_all_int(data['int_Content_splited_morph'])


In [13]:
len(all_txt_as_int), len(all_txt_as_int_morph)

(27414, 27414)

In [14]:
# The maximum length sentence you want for a single input in characters
seq_length = 10
examples_per_epoch = len(all_txt_as_int)//(seq_length+1)

# Create training examples / targets

dataset1 = tf.data.Dataset.from_tensor_slices(all_txt_as_int)
dataset2 = tf.data.Dataset.from_tensor_slices(all_txt_as_int_morph)

sequences1 = dataset1.batch(seq_length+1, drop_remainder=True)
sequences2 = dataset2.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def split_input_morph(chunk):
    input_text = chunk[:-1]
    return input_text




dataset1 = sequences1.map(split_input_target)
dataset2 = sequences2.map(split_input_morph)





dataset = tf.data.Dataset.zip((dataset1, dataset2))
dataset

<ZipDataset shapes: (((10,), (10,)), (10,)), types: ((tf.int32, tf.int32), tf.int32)>

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: (((64, 10), (64, 10)), (64, 10)), types: ((tf.int32, tf.int32), tf.int32)>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])
    
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size))(x)
    print(x.shape)

    
    
    print('==================================================')
    
    
    
    inputs_morph = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    xm =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs_morph)
    print(xm.shape)
    x1m = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(xm)
    xm = tf.keras.layers.concatenate([xm,x1m], axis=-1)
    
    print(xm.shape)
    x2m = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(xm)
    xm = tf.keras.layers.add([xm,x2m])
    
    print(xm.shape)
    x3m = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(xm)   
    
    xm = tf.keras.layers.add([xm,x3m])
    
    
    
    xm = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size))(xm)
    print(xm.shape)    
    
    
    
    x = tf.keras.layers.Concatenate()([x, xm])      
    
    x = tf.keras.layers.Dense(vocab_size)(x)    
    
    
    model =tf.keras. Model(inputs=[inputs, inputs_morph], outputs=x)
        
    return model

In [18]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 8427)
(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 8427)


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (64, None, 256)      2157312     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (64, None, 256)      2157312     input_2[0][0]                    
______________________________________________________________________________________________

In [20]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [21]:
model.compile(optimizer='adam', loss=loss)

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=10,
    save_weights_only=True)

In [23]:
EPOCHS = 11

In [24]:
for i in range(EPOCHS):
    model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])
    model.reset_states()

38/38 [==============================] - 8s 218ms/step - loss: 0.0133


In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_1'

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 8427)
(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 8427)


In [27]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of Words to generate
    num_generate = 30

    
    start_string = exclude_punctuation(start_string)
    #print(start_string)
    start_string_asis = preprocess_text(start_string, morph = False)
    start_string_morph = preprocess_text(start_string, morph = True)    

      
    
    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string_asis]
    input_eval = tf.expand_dims(input_eval, 0)
    
    input_eval_morph = [word2idx_morph[s] for s in start_string_morph]
    input_eval_morph = tf.expand_dims(input_eval_morph, 0)
     
    

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model([input_eval, input_eval_morph])
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        a=preprocess_text(idx2word[predicted_id], morph = True)
        predicted_id_morph=word2idx_morph[a[0]]
        
        
        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)       
        input_eval_morph = tf.expand_dims([predicted_id_morph], 0)
                
        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [28]:
print(generate_text(model, start_string="Святой исполненной мечты "))

Святой исполненной мечты жестокая высокие вечор братцы благословенное дрогнет пепел обряд конюшня дружба ближний безбожно дрогнет незваных звала горькую гремел вновь бытописания блистает беспечна безбожно ветреное вперил замечая гостях вменяя былей близ беседуют


In [29]:
print(generate_text(model, start_string="Мой дядя самых честных правил? "))

Мой дядя самых честных правил людьми пламенный красотой золотой высока беспечна безбожно мрак завез оплошного бровями веселья бородатый благоразумной никогда знал гремел вновь бытописания блистает беспечна безбожно ветреное петербургский до воспела веков дружеский заняв дидло


In [30]:
print(generate_text(model, start_string="Мой дядя самых честных правил \n"))

Мой дядя самых честных правил  
 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  



# Вывод

Качество генерируемого текста оставляет желать лучшего. Использование двух "голов" не привело к улучшению качества текста.